In [7]:
import pandas as pd
import os
import re
from tqdm.auto import tqdm

tqdm.pandas()

# --- 1. 設定 ---
INPUT_FILE = "data/processed/training_dataset_abstract_cleaned.csv" # 元のファイル
OUTPUT_FILE = "data/processed/training_dataset_abstract_cleaned_v2.csv" # 新しい出力ファイル名

# アブストラクトの終わり（＝本文の始まり）を示す可能性が非常に高いキーワード
# （大文字と小文字を区別しないように、すべて小文字で定義）
STOP_WORDS = [
    'introduction',
    'keywords',
    'key words',
    'references',
    'acknowledgments',
    'acknowledgements',
    'bibliography',
    'pubmed abstract',
    'publisher full text',
    'full text'
]

# 正規表現パターンを作成 ( | で区切る)
# \b は単語の境界を意味し、"introduction" が "subintroduction" にマッチしないようにする
STOP_PATTERN = re.compile(r'\b(' + '|'.join(STOP_WORDS) + r')\b', re.IGNORECASE)

# URLを検出する正規表現 (簡略版)
URL_PATTERN = re.compile(r'https?://\S+|www\.\S+')

# メールアドレスを検出する正規表現
EMAIL_PATTERN = re.compile(r'\S*@\S*\s?')

# 非ASCII文字を検出する正規表現 (ASCII文字以外すべて)
NON_ASCII_PATTERN = re.compile(r'[^\x00-\x7F]+')

# 特殊記号を検出する正規表現 (句読点と一般的な記号は残し、それ以外を削除)
# 今回は極端な特殊記号（例：絵文字、一部の数学記号など）に限定し、
# 句読点（,.!?"'）やハイフン、括弧などは残す方針で
SPECIAL_CHARS_PATTERN = re.compile(r'[^\w\s\.\,\!\?\-\'\(\)\[\]\{\}\<\>\/\=\+\*\%]') # 基本的な記号は残す
# もし、より厳密にアルファベットと数字、基本的な句読点のみにしたい場合は以下
# SPECIAL_CHARS_PATTERN = re.compile(r'[^a-zA-Z0-9\s\.\,\!\?\-:\'\(\)\{\}\[\]]')


print("Settings and cleaning patterns defined.")

Settings and cleaning patterns defined.


In [8]:
# --- 2. クリーニング関数の定義 ---

def comprehensive_clean_abstract(text):
    """
    テキストを受け取り、以下のクリーニングを行う
    1. 末尾の本文ノイズの切り捨て
    2. URLの削除
    3. メールアドレスの削除
    4. 非ASCII文字の削除
    5. 特殊記号の削除 (一部残す)
    """
    if not isinstance(text, str):
        return text
    
    # 1. 末尾の本文ノイズの切り捨て (STOP_PATTERNを使用)
    match = STOP_PATTERN.search(text)
    if match:
        text = text[:match.start()]
    
    # 2. URLの削除
    text = URL_PATTERN.sub('', text)
    
    # 3. メールアドレスの削除
    text = EMAIL_PATTERN.sub('', text)

    # 4. 非ASCII文字の削除
    text = NON_ASCII_PATTERN.sub('', text)
    
    # 5. 特殊記号の削除 (一般的な句読点や記号は残す)
    text = SPECIAL_CHARS_PATTERN.sub('', text)
    
    # 連続する空白文字を1つにまとめる
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# テスト実行
test_text = "This is abstract. See more at https://example.com. Contact us at user@domain.com. We found α=0.05. Some strange char: ™, ©, 『Test』. This is body..."
print(f"Original: {test_text}")
print(f"Cleaned:  {comprehensive_clean_abstract(test_text)}")

test_text_no_body = "This is a clean abstract. No body here. 123.-_()[]{}<>/?=+%*"
print(f"\nOriginal (no body): {test_text_no_body}")
print(f"Cleaned (no body):  {comprehensive_clean_abstract(test_text_no_body)}")

Original: This is abstract. See more at https://example.com. Contact us at user@domain.com. We found α=0.05. Some strange char: ™, ©, 『Test』. This is body...
Cleaned:  This is abstract. See more at Contact us at We found =0.05. Some strange char , , Test. This is body...

Original (no body): This is a clean abstract. No body here. 123.-_()[]{}<>/?=+%*
Cleaned (no body):  This is a clean abstract. No body here. 123.-_()[]{}<>/?=+%*


In [9]:
# --- 3. データセットの読み込みと前処理の実行 ---
print(f"Loading dataset from {INPUT_FILE}...")
df = pd.read_csv(INPUT_FILE)
df = df.dropna(subset=['abstract_a', 'abstract_b'])

print("Applying comprehensive cleaning function to 'abstract_a'...")
df['abstract_a_cleaned'] = df['abstract_a'].progress_apply(comprehensive_clean_abstract)

print("Applying comprehensive cleaning function to 'abstract_b'...")
df['abstract_b_cleaned'] = df['abstract_b'].progress_apply(comprehensive_clean_abstract)

print("Cleaning complete.")

Loading dataset from data/processed/training_dataset_abstract_cleaned.csv...
Applying comprehensive cleaning function to 'abstract_a'...


  0%|          | 0/34625 [00:00<?, ?it/s]

Applying comprehensive cleaning function to 'abstract_b'...


  0%|          | 0/34625 [00:00<?, ?it/s]

Cleaning complete.


In [10]:
# --- 4. 最終データセットの保存 ---

# 元のアブストラクトを削除し、クリーニング後のものだけを残す
df_final = df.drop(columns=['abstract_a', 'abstract_b'])
df_final = df_final.rename(columns={
    'abstract_a_cleaned': 'abstract_a',
    'abstract_b_cleaned': 'abstract_b'
})

# 必要なカラムの順序に並び替え
df_final = df_final[['abstract_a', 'abstract_b', 'label', 'data_paper_doi']]

print(f"Saving cleaned dataset to {OUTPUT_FILE}...")
df_final.to_csv(OUTPUT_FILE, index=False)
print("Save complete.")

Saving cleaned dataset to data/processed/training_dataset_abstract_cleaned_v2.csv...
Save complete.


In [ ]:
import pandas as pd
import os
from tqdm.auto import tqdm

tqdm.pandas()

# --- 1. 設定 ---
# 読み込むクリーニング済みファイル
INPUT_FILE = "data/processed/training_dataset_abstract_cleaned_v2.csv" 
# 最終的な訓練データセット
OUTPUT_FILE = "data/processed/training_dataset_abstract_cleaned_v3.csv" 

# ▼▼▼ 最終的な文字数上限（ハードリミット）を設定 ▼▼▼
# 75%が約1500文字、一般的な上限が2500文字であることから、
# 3000文字あたりを「絶対にノイズ」と判断する閾値として設定
MAX_CHAR_LIMIT = 3500

print(f"Settings defined. All abstracts will be truncated to {MAX_CHAR_LIMIT} characters.")

Settings defined. All abstracts will be truncated to 3500 characters.


In [14]:
# --- 2. クリーニング関数の定義 ---

def truncate_text(text):
    """
    テキストを受け取り、先頭から MAX_CHAR_LIMIT 文字だけを残す
    """
    if not isinstance(text, str):
        return text
    
    # Pythonのスライス機能を使って、先頭からMAX_CHAR_LIMIT文字目までを取得
    return text[:MAX_CHAR_LIMIT]

# テスト実行
test_text = "A" * (MAX_CHAR_LIMIT + 5)
print(f"Original Length: {len(test_text)}")
print(f"Cleaned Length:  {len(truncate_text(test_text))}")

Original Length: 3505
Cleaned Length:  3500


In [15]:
# --- 3. データセットの読み込みと前処理の実行 ---
print(f"Loading dataset from {INPUT_FILE}...")
df = pd.read_csv(INPUT_FILE)
df = df.dropna(subset=['abstract_a', 'abstract_b'])

print("Applying truncation to 'abstract_a'...")
df['abstract_a_cleaned'] = df['abstract_a'].progress_apply(truncate_text)

print("Applying truncation to 'abstract_b'...")
df['abstract_b_cleaned'] = df['abstract_b'].progress_apply(truncate_text)

print("Truncation complete.")

Loading dataset from data/processed/training_dataset_abstract_cleaned_v2.csv...
Applying truncation to 'abstract_a'...


  0%|          | 0/34624 [00:00<?, ?it/s]

Applying truncation to 'abstract_b'...


  0%|          | 0/34624 [00:00<?, ?it/s]

Truncation complete.


In [16]:
# --- 4. 最終データセットの保存 ---

# 元のアブストラクトを削除し、クリーニング後のものだけを残す
df_final = df.drop(columns=['abstract_a', 'abstract_b', 'char_len_a', 'char_len_b', 'max_char_len'], errors='ignore') # 以前の分析列も削除
df_final = df_final.rename(columns={
    'abstract_a_cleaned': 'abstract_a',
    'abstract_b_cleaned': 'abstract_b'
})

# 必要なカラムの順序に並び替え
df_final = df_final[['abstract_a', 'abstract_b', 'label', 'data_paper_doi']]

print(f"Saving final cleaned dataset to {OUTPUT_FILE}...")
df_final.to_csv(OUTPUT_FILE, index=False)
print("Save complete.")

# 念のため、処理後の最大文字数を確認
print("\nMax length in final 'abstract_a':", df_final['abstract_a'].str.len().max())
print("Max length in final 'abstract_b':", df_final['abstract_b'].str.len().max())

Saving final cleaned dataset to data/processed/training_dataset_abstract_v3.csv...
Save complete.

Max length in final 'abstract_a': 3500
Max length in final 'abstract_b': 3500
